In [1]:
import torch
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import transformers
from transformers import BertTokenizer, BertForSequenceClassification
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = '/content/drive/My Drive/LIN371/classification_only_model'

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # load model
# model = BertForSequenceClassification.from_pretrained(model_path)
# model.to(device)
# model.eval()
# model.zero_grad()

# # load tokenizer
# tokenizer = BertTokenizer.from_pretrained(model_path)

In [3]:
def predict(model, inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(
        inputs,
        token_type_ids=token_type_ids,
        position_ids=position_ids,
        attention_mask=attention_mask
    )
    return output.logits  # For sequence classification


In [4]:
inputs = tokenizer(
    ["crazy big body cool we gonna "],
    return_tensors="pt",
    truncation=True,
    max_length=512,
    padding='max_length',
)

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

replace_tokens = ['original']

for tokenized_sent in input_ids:
  for i, token in enumerate(tokenized_sent):
    if token not in [tokenizer.pad_token_id, tokenizer.cls_token_id, tokenizer.sep_token_id]:
      new_sent = tokenized_sent.clone().detach()
      new_sent[i] = tokenizer.mask_token_id
      input_ids = torch.cat((input_ids, new_sent.unsqueeze(0)), dim=0)
      replace_tokens.append(tokenizer.decode(token))

attention_mask = (input_ids != tokenizer.pad_token_id).long()
print(attention_mask.shape)
print(input_ids.shape)

torch.Size([7, 512])
torch.Size([7, 512])


In [5]:
pred = predict(inputs=input_ids.to(device), attention_mask=attention_mask.to(device))
pred

TypeError: predict() missing 1 required positional argument: 'model'

In [6]:
import torch.nn.functional as F

probabilities = F.softmax(pred, dim=-1)

print(probabilities)

NameError: name 'pred' is not defined

In [ ]:
differences = (probabilities - probabilities[0])[:,0]
differences

tensor([0.0000, 0.0009, 0.0016, 0.2570, 0.0005, 0.0044, 0.0015],
       grad_fn=<SelectBackward0>)

In [ ]:
max_len = max(len(token) for token in replace_tokens)

print(f"{'token':^{max_len}} |",
      f"{'class 0 proba':^{13}} |",
      f"{'class 1 proba':^{13}} |",
      f"{'change (class 0)':^{16}} |")
for probability, token, diff in zip(probabilities, replace_tokens, differences):
  class0 = probability[0]
  class1 = probability[1]
  print(f'{token:^{max_len}} |',
        f'{round(class0.item(), 3):^{13}} |',
        f'{round(class1.item(), 3):^{13}} |',
        f'{round(diff.item(), 3):^{16}} |')

 token   | class 0 proba | class 1 proba | change (class 0) |
original |     0.001     |     0.999     |       0.0        |
 crazy   |     0.002     |     0.998     |      0.001       |
  big    |     0.003     |     0.997     |      0.002       |
  body   |     0.258     |     0.742     |      0.257       |
  cool   |     0.001     |     0.999     |       0.0        |
   we    |     0.005     |     0.995     |      0.004       |
 gonna   |     0.003     |     0.997     |      0.002       |


In [4]:
import torch.nn.functional as F

def perturb_sent(model, tokenizer, sent):
  inputs = tokenizer(
    [sent],
    return_tensors="pt",
    truncation=True,
    max_length=512,
    padding='max_length',
)

  input_ids = inputs["input_ids"]
  attention_mask = inputs["attention_mask"]

  replace_tokens = ['unmasked']

  for tokenized_sent in input_ids:
    for i, token in enumerate(tokenized_sent):
      if token not in [tokenizer.pad_token_id, tokenizer.cls_token_id, tokenizer.sep_token_id]:
        new_sent = tokenized_sent.clone().detach()
        new_sent[i] = tokenizer.mask_token_id
        input_ids = torch.cat((input_ids, new_sent.unsqueeze(0)), dim=0)
        replace_tokens.append(tokenizer.decode(token))

  attention_mask = (input_ids != tokenizer.pad_token_id).long()

  pred = predict(model, inputs=input_ids.to(device), attention_mask=attention_mask.to(device))

  probabilities = F.softmax(pred, dim=-1)

  differences = (probabilities - probabilities[0])[:,0]

  max_len = max(len(token) for token in replace_tokens)

  print('sentence:', sent)
  print(f"{'token':^{max_len}} |",
        f"{'class 0 proba':^{13}} |",
        f"{'class 1 proba':^{13}} |",
        f"{'change (class 0)':^{16}} |")
  for probability, token, diff in zip(probabilities, replace_tokens, differences):
    class0 = probability[0]
    class1 = probability[1]
    print(f'{token:^{max_len}} |',
          f'{round(class0.item(), 3):^{13}} |',
          f'{round(class1.item(), 3):^{13}} |',
          f'{round(diff.item(), 3):^{16}} |')

  return probabilities, replace_tokens, differences, abs(differences)


In [5]:
# read predictions
predicitons = pd.read_csv("/content/drive/My Drive/LIN371/predictions.csv")
predicitons.head()

,Unnamed: 0,text,label,new_label,prediction
0,0,Those pussy lips need more cleaning with my to...,1,explicit_source_has_explicit_words,1
1,1,"I have choices for you. Choice seating, at that",1,explicit_source_no_explicit_words,1
2,2,I want to finish.,1,explicit_source_no_explicit_words,0
3,3,"Oh it Will, one way or an other 😉",1,explicit_source_no_explicit_words,1
4,4,"No need to thank me, thank you so much for sha...",1,explicit_source_no_explicit_words,0


In [8]:
new_predictions = pd.read_csv("/content/drive/My Drive/LIN371/predictions_mlm_tuned.csv")
new_predictions.head()

,Unnamed: 0,text,label,new_label,prediction
0,0,Those pussy lips need more cleaning with my to...,1,explicit_source_has_explicit_words,1
1,1,"I have choices for you. Choice seating, at that",1,explicit_source_no_explicit_words,1
2,2,I want to finish.,1,explicit_source_no_explicit_words,1
3,3,"Oh it Will, one way or an other 😉",1,explicit_source_no_explicit_words,1
4,4,"No need to thank me, thank you so much for sha...",1,explicit_source_no_explicit_words,1


In [6]:
# get some examples where pred is 1

pred_1 = predicitons[predicitons['prediction'] == 1].sample(5, random_state=5)
texts = pred_1['text'].tolist()
texts

['Looks wet and ready',
 'Come here you cuddly little rascal',
 'Exactly what I need after a long day at work.',
 'man i love a Bush! u are stunning',
 'Mmmmm mmmm mmm\n']

In [15]:
new_predictions[new_predictions['text'].isin(texts)]

,Unnamed: 0,text,label,new_label,prediction
128,128,Mmmmm mmmm mmm\n,1,explicit_source_no_explicit_words,1
365,365,Come here you cuddly little rascal,0,control_source_no_explicit_words,0
654,654,Looks wet and ready,1,explicit_source_no_explicit_words,1
2283,2283,Exactly what I need after a long day at work.,1,explicit_source_no_explicit_words,1
2544,2544,man i love a Bush! u are stunning,1,explicit_source_no_explicit_words,1


In [10]:
# get some examples where pred is 1

pred_1 = predicitons[predicitons['prediction'] == 1].sample(5, random_state=5)
texts = pred_1['text'].tolist()
texts

['Looks wet and ready',
 'Come here you cuddly little rascal',
 'Exactly what I need after a long day at work.',
 'man i love a Bush! u are stunning',
 'Mmmmm mmmm mmm\n']

In [10]:
proba0, tokens0, diffs0, absdiffs0 = perturb_sent(model, tokenizer, texts[0])
proba1, tokens1, diffs1, absdiffs1 = perturb_sent(model, tokenizer, texts[1])
proba2, tokens2, diffs2, absdiffs2 = perturb_sent(model, tokenizer, texts[2])
proba3, tokens3, diffs3, absdiffs3 = perturb_sent(model, tokenizer, texts[3])
proba4, tokens4, diffs4, absdiffs4 = perturb_sent(model, tokenizer, texts[4])

sentence: Looks wet and ready
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.001     |     0.999     |       0.0        |
 looks   |     0.004     |     0.996     |      0.002       |
  wet    |     0.029     |     0.971     |      0.028       |
  and    |     0.002     |     0.998     |       0.0        |
 ready   |     0.015     |     0.985     |      0.013       |
sentence: Come here you cuddly little rascal
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.34      |     0.66      |       0.0        |
  come   |     0.682     |     0.318     |      0.342       |
  here   |     0.199     |     0.801     |      -0.14       |
  you    |     0.391     |     0.609     |      0.051       |
   cu    |     0.723     |     0.277     |      0.383       |
  ##dd   |     0.518     |     0.482     |      0.178       |
  ##ly   |     0.464     |     0.536     |      0.124       |
 little  |     0.409     |     0.591     |      0.069    

In [7]:
# load the mlm model
mlm_model = BertForSequenceClassification.from_pretrained('/content/drive/My Drive/LIN371/bert-base-uncased-mlm-classifier')
mlm_tokenizer = BertTokenizer.from_pretrained('/content/drive/My Drive/LIN371/bert-base-uncased-mlm-classifier')
mlm_model.to(device)
mlm_model.eval()
mlm_model.zero_grad()


In [8]:
mlm_proba0, mlm_tokens0, mlm_diffs0, mlm_absdiffs0 = perturb_sent(mlm_model, mlm_tokenizer, texts[0])
mlm_proba1, mlm_tokens1, mlm_diffs1, mlm_absdiffs1 = perturb_sent(mlm_model, mlm_tokenizer, texts[1])
mlm_proba2, mlm_tokens2, mlm_diffs2, mlm_absdiffs2 = perturb_sent(mlm_model, mlm_tokenizer, texts[2])
mlm_proba3, mlm_tokens3, mlm_diffs3, mlm_absdiffs3 = perturb_sent(mlm_model, mlm_tokenizer, texts[3])
mlm_proba4, mlm_tokens4, mlm_diffs4, mlm_absdiffs4 = perturb_sent(mlm_model, mlm_tokenizer, texts[4])

sentence: Looks wet and ready
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.018     |     0.982     |       0.0        |
 looks   |     0.049     |     0.951     |      0.031       |
  wet    |     0.076     |     0.924     |      0.058       |
  and    |     0.045     |     0.955     |      0.027       |
 ready   |     0.16      |     0.84      |      0.142       |
sentence: Come here you cuddly little rascal
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.565     |     0.435     |       0.0        |
  come   |     0.739     |     0.261     |      0.174       |
  here   |     0.565     |     0.435     |       0.0        |
  you    |     0.649     |     0.351     |      0.084       |
   cu    |     0.584     |     0.416     |      0.018       |
  ##dd   |     0.559     |     0.441     |      -0.006      |
  ##ly   |     0.608     |     0.392     |      0.043       |
 little  |     0.682     |     0.318     |      0.117    

In [22]:
diffs0.shape

torch.Size([5])

In [46]:
# make dfs
df0 = pd.DataFrame({'token':tokens0,
                    'perturb_proba0': proba0[:,0].cpu().detach().numpy(),
                    'perturb_proba0_change': diffs0.cpu().detach().numpy()})
df1 = pd.DataFrame({'token':tokens1,
                    'perturb_proba0': proba1[:,0].cpu().detach().numpy(),
                    'perturb_proba0_change': diffs1.cpu().detach().numpy()})
df2 = pd.DataFrame({'token':tokens2,
                    'perturb_proba0': proba2[:,0].cpu().detach().numpy(),
                    'perturb_proba0_change': diffs2.cpu().detach().numpy()})
df3 = pd.DataFrame({'token':tokens3,
                    'perturb_proba0': proba3[:,0].cpu().detach().numpy(),
                    'perturb_proba0_change': diffs3.cpu().detach().numpy()})
df4 = pd.DataFrame({'token':tokens4,
                    'perturb_proba0': proba4[:,0].cpu().detach().numpy(),
                    'perturb_proba0_change': diffs4.cpu().detach().numpy()})

In [45]:
update_text0_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/text0_df.csv')
update_text1_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/text1_df.csv')
update_text2_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/text2_df.csv')
update_text3_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/text3_df.csv')
update_text4_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/text4_df.csv')

In [53]:

result0 = pd.merge( update_text0_df, df0, on='token', how='outer')
custom_order = ['[CLS]'] + tokens0[1:] + ['[SEP]'] + ['unmasked']
result0['key'] = pd.Categorical(result0['token'], categories=custom_order, ordered=True)
result0 = result0.sort_values('key').reset_index(drop=True)

result1 = pd.merge( update_text1_df, df1, on='token', how='outer')
custom_order = ['[CLS]'] + tokens1[1:] + ['[SEP]'] + ['unmasked']
result1['key'] = pd.Categorical(result1['token'], categories=custom_order, ordered=True)
result1 = result1.sort_values('key').reset_index(drop=True)

result2 = pd.merge( update_text2_df, df2, on='token', how='outer')
custom_order = ['[CLS]'] + tokens2[1:] + ['[SEP]'] + ['unmasked']
result2['key'] = pd.Categorical(result2['token'], categories=custom_order, ordered=True)
result2 = result2.sort_values('key').reset_index(drop=True)

result3 = pd.merge( update_text3_df, df3, on='token', how='outer')
custom_order = ['[CLS]'] + tokens3[1:] + ['[SEP]'] + ['unmasked']
result3['key'] = pd.Categorical(result3['token'], categories=custom_order, ordered=True)
result3 = result3.sort_values('key').reset_index(drop=True)

result4 = pd.merge( update_text4_df, df4, on='token', how='outer')
# custom_order = ['[CLS]'] + tokens4[1:] + ['[SEP]'] + ['unmasked']
# result4['key'] = pd.Categorical(result4['token'], categories=custom_order, ordered=True)
# result4 = result4.sort_values('key').reset_index(drop=True)





,Unnamed: 0,token,simple_l2,mlm_simple_l2,int_grad,mlm_int_grad,perturb_proba0,perturb_proba0_change
0,4.0,##m,0.562105,1.337196,0.179376,0.394852,0.001674,0.000797
1,2.0,##mm,0.899090,1.768558,0.154570,0.331149,0.002322,0.001444
2,0.0,[CLS],0.388557,0.984441,0.061301,0.004525,NaN,NaN
3,6.0,[SEP],0.737157,1.498512,0.732653,0.237356,NaN,NaN
4,1.0,mmm,0.989370,1.918854,0.181136,0.340139,0.001796,0.000918
5,1.0,mmm,0.989370,1.918854,0.181136,0.340139,0.002075,0.001198
6,1.0,mmm,0.989370,1.918854,0.181136,0.340139,0.003862,0.002984
7,3.0,mmm,0.813938,2.059563,0.244204,0.441549,0.001796,0.000918
8,3.0,mmm,0.813938,2.059563,0.244204,0.441549,0.002075,0.001198
9,3.0,mmm,0.813938,2.059563,0.244204,0.441549,0.003862,0.002984


In [9]:
# make dfs
df0 = pd.DataFrame({'token':mlm_tokens0,
                    'mlm_perturb_proba0': mlm_proba0[:,0].cpu().detach().numpy(),
                    'mlm_perturb_proba0_change': mlm_diffs0.cpu().detach().numpy()})
df1 = pd.DataFrame({'token':mlm_tokens1,
                    'mlm_perturb_proba0': mlm_proba1[:,0].cpu().detach().numpy(),
                    'mlm_perturb_proba0_change': mlm_diffs1.cpu().detach().numpy()})
df2 = pd.DataFrame({'token':mlm_tokens2,
                    'mlm_perturb_proba0': mlm_proba2[:,0].cpu().detach().numpy(),
                    'mlm_perturb_proba0_change': mlm_diffs2.cpu().detach().numpy()})
df3 = pd.DataFrame({'token':mlm_tokens3,
                    'mlm_perturb_proba0': mlm_proba3[:,0].cpu().detach().numpy(),
                    'mlm_perturb_proba0_change': mlm_diffs3.cpu().detach().numpy()})
df4 = pd.DataFrame({'token':mlm_tokens4,
                    'mlm_perturb_proba0': mlm_proba4[:,0].cpu().detach().numpy(),
                    'mlm_perturb_proba0_change': mlm_diffs4.cpu().detach().numpy()})

In [10]:
df1

,token,mlm_perturb_proba0,mlm_perturb_proba0_change
0,unmasked,0.565065,0.000000
1,come,0.739240,0.174175
2,here,0.565381,0.000316
3,you,0.648578,0.083513
4,cu,0.583514,0.018449
5,##dd,0.558675,-0.006389
6,##ly,0.607958,0.042894
7,little,0.681644,0.116579
8,ras,0.487899,-0.077166
9,##cal,0.442191,-0.122874


In [11]:
update_text0_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/result0.csv')
update_text1_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/result1.csv')
update_text2_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/result2.csv')
update_text3_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/result3.csv')
update_text4_df = pd.read_csv('/content/drive/My Drive/LIN371/example_sent/result4.csv')

In [12]:
update_text0_df

,Unnamed: 0.1,Unnamed: 0,token,simple_l2,mlm_simple_l2,int_grad,mlm_int_grad,perturb_proba0,perturb_proba0_change,key
0,0,0.0,[CLS],0.667221,0.913147,0.089534,0.083628,NaN,NaN,[CLS]
1,1,1.0,looks,3.247836,4.324699,-0.067812,0.039359,0.003636,0.002189,looks
2,2,2.0,wet,2.730376,4.984249,0.673710,0.443098,0.029337,0.027890,wet
3,3,3.0,and,1.078880,1.572279,0.133770,0.374883,0.001586,0.000139,and
4,4,4.0,ready,2.421854,3.655497,0.520036,0.806675,0.014627,0.013181,ready
5,5,5.0,[SEP],1.365179,2.110789,0.495145,-0.062130,NaN,NaN,[SEP]
6,6,NaN,unmasked,NaN,NaN,NaN,NaN,0.001447,0.000000,unmasked


In [17]:
result0 = pd.merge( update_text0_df, df0, on='token', how='left')
result1 = pd.merge( update_text1_df, df1, on='token', how='left')
result2 = pd.merge( update_text2_df, df2, on='token', how='left')
result3 = pd.merge( update_text3_df, df3, on='token', how='left')
result4 = pd.merge( update_text4_df, df4, on='token', how='left')
result3

,Unnamed: 0.1,Unnamed: 0,token,simple_l2,mlm_simple_l2,int_grad,mlm_int_grad,perturb_proba0,perturb_proba0_change,key,mlm_perturb_proba0,mlm_perturb_proba0_change
0,0,0.0,[CLS],0.108779,0.260486,0.030687,0.012645,NaN,NaN,[CLS],NaN,NaN
1,1,1.0,man,0.283163,0.581357,-0.074261,-0.079672,0.000687,-0.000119,man,0.002232,-0.000073
2,2,2.0,i,0.157900,0.271053,0.063783,0.034830,0.000764,-0.000042,i,0.002264,-0.000041
3,3,3.0,love,0.224341,0.424006,0.368018,0.355517,0.000745,-0.000061,love,0.002550,0.000246
4,4,4.0,a,0.226504,0.276085,0.193575,0.179483,0.000726,-0.000080,a,0.002399,0.000095
5,5,5.0,bush,0.671435,0.923071,0.220481,0.060750,0.000811,0.000004,bush,0.002272,-0.000032
6,6,6.0,!,0.275994,0.560980,0.131700,0.236318,0.000795,-0.000011,!,0.003024,0.000720
7,7,7.0,u,0.500622,1.403063,0.240636,0.393314,0.001397,0.000591,u,0.004140,0.001836
8,8,8.0,are,0.237376,0.582913,0.043333,-0.041559,0.001080,0.000274,are,0.002857,0.000552
9,9,9.0,stunning,0.695976,1.127330,0.232879,0.571397,0.006175,0.005369,stunning,0.024346,0.022042


In [18]:
result0.to_csv('/content/drive/My Drive/LIN371/example_sent/result0.csv')
result1.to_csv('/content/drive/My Drive/LIN371/example_sent/result1.csv')
result2.to_csv('/content/drive/My Drive/LIN371/example_sent/result2.csv')
result3.to_csv('/content/drive/My Drive/LIN371/example_sent/result3.csv')
result4.to_csv('/content/drive/My Drive/LIN371/example_sent/result4.csv')

In [ ]:
# load some predicted examples
df = pd.read_csv('/content/drive/My Drive/LIN371/predictions.csv')
df.head()


,Unnamed: 0,text,label,new_label,prediction
0,0,Those pussy lips need more cleaning with my to...,1,explicit_source_has_explicit_words,1
1,1,"I have choices for you. Choice seating, at that",1,explicit_source_no_explicit_words,1
2,2,I want to finish.,1,explicit_source_no_explicit_words,0
3,3,"Oh it Will, one way or an other 😉",1,explicit_source_no_explicit_words,1
4,4,"No need to thank me, thank you so much for sha...",1,explicit_source_no_explicit_words,0


In [ ]:
correct_exp_source_exp_words = df[(df['new_label'] == 'explicit_source_has_explicit_words') & (df['prediction'] == df['label'])]
correct_exp_source_exp_words_sample = correct_exp_source_exp_words.sample(3, random_state=20)
correct_exp_source_exp_words_sample

,Unnamed: 0,text,label,new_label,prediction
773,773,Yes but id also like to lick you prior to fill...,1,explicit_source_has_explicit_words,1
4090,4090,Would love to fill you deep. I want my balls s...,1,explicit_source_has_explicit_words,1
2652,2652,"you have an amazing ass, don't know why you do...",1,explicit_source_has_explicit_words,1


In [ ]:
for sent in correct_exp_source_exp_words_sample['text']:
  perturb_sent(sent)
  print()

sentence: Yes but id also like to lick you prior to filling you up. You have a fine ass body
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.001     |     0.999     |       0.0        |
  yes    |     0.002     |     0.998     |      0.001       |
  but    |     0.001     |     0.999     |       0.0        |
   id    |     0.001     |     0.999     |       0.0        |
  also   |     0.001     |     0.999     |       0.0        |
  like   |     0.001     |     0.999     |       0.0        |
   to    |     0.001     |     0.999     |       0.0        |
  lick   |     0.001     |     0.999     |       -0.0       |
  you    |     0.002     |     0.998     |       0.0        |
 prior   |     0.001     |     0.999     |       -0.0       |
   to    |     0.001     |     0.999     |       -0.0       |
filling  |     0.001     |     0.999     |       0.0        |
  you    |     0.001     |     0.999     |       -0.0       |
   up    |     0.001     |     0.999   

In [ ]:
# more interesting example, when the new label is explicit_source_no_explicit_words
correct_exp_source_no_exp_words = df[(df['new_label'] == 'explicit_source_no_explicit_words') & (df['prediction'] == df['label'])]
correct_exp_source_no_exp_words_sample = correct_exp_source_no_exp_words.sample(3, random_state=100)
correct_exp_source_no_exp_words_sample

,Unnamed: 0,text,label,new_label,prediction
3727,3727,Looks good enough to eat !\n,1,explicit_source_no_explicit_words,1
1219,1219,Gorgeous! Love the view! Perhaps some hungry l...,1,explicit_source_no_explicit_words,1
1058,1058,He is lucky to have you,1,explicit_source_no_explicit_words,1


In [ ]:
for sent in correct_exp_source_no_exp_words_sample['text']:
  print(len(sent.split()))

6
16
14


In [ ]:
for sent in correct_exp_source_no_exp_words_sample['text']:
  perturb_sent(sent)
  print()

sentence: Looks good enough to eat !

 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.002     |     0.998     |       0.0        |
 looks   |     0.048     |     0.952     |      0.046       |
  good   |     0.006     |     0.994     |      0.004       |
 enough  |     0.001     |     0.999     |      -0.001      |
   to    |     0.001     |     0.999     |      -0.001      |
  eat    |     0.001     |     0.999     |      -0.001      |
   !     |     0.004     |     0.996     |      0.002       |

sentence: Gorgeous! Love the view! Perhaps some hungry licking?
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.002     |     0.998     |       0.0        |
gorgeous |     0.001     |     0.999     |      -0.001      |
   !     |     0.001     |     0.999     |       -0.0       |
  love   |     0.001     |     0.999     |      -0.001      |
  the    |     0.001     |     0.999     |      -0.001      |
  view   |     0.001     |   

In [ ]:
# incorrect prediction
incorrect_exp_source_exp_words = df[(df['new_label'] == 'explicit_source_has_explicit_words') & (df['prediction'] != df['label'])]
incorrect_exp_source_exp_words_sample = incorrect_exp_source_exp_words.sample(3, random_state=80)
incorrect_exp_source_exp_words_sample

,Unnamed: 0,text,label,new_label,prediction
2549,2549,People on your snapchat are lucky as fuck,1,explicit_source_has_explicit_words,0
2022,2022,Oh my fucking god. I have to clear my schedule,1,explicit_source_has_explicit_words,0
3763,3763,"That’s called an asshole, mate.",1,explicit_source_has_explicit_words,0


In [ ]:
for sent in incorrect_exp_source_exp_words_sample['text']:
  print(sent)

People on your snapchat are lucky as fuck
Oh my fucking god. I have to clear my schedule
That’s called an asshole, mate.


In [ ]:
for sent in incorrect_exp_source_exp_words_sample['text']:
  perturb_sent(sent)
  print()

sentence: People on your snapchat are lucky as fuck
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.859     |     0.141     |       0.0        |
 people  |     0.477     |     0.523     |      -0.382      |
   on    |     0.817     |     0.183     |      -0.043      |
  your   |     0.854     |     0.146     |      -0.005      |
  snap   |     0.777     |     0.223     |      -0.082      |
 ##cha   |     0.81      |     0.19      |      -0.049      |
  ##t    |     0.835     |     0.165     |      -0.024      |
  are    |     0.808     |     0.192     |      -0.051      |
 lucky   |     0.876     |     0.124     |      0.017       |
   as    |     0.869     |     0.131     |      0.009       |
  fuck   |     0.869     |     0.131     |       0.01       |

sentence: Oh my fucking god. I have to clear my schedule
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.639     |     0.361     |       0.0        |
   oh    |     0.731  

In [ ]:
# incorrect prediction for no explicit words
incorrect_exp_source_no_exp_words = df[(df['new_label'] == 'explicit_source_no_explicit_words') & (df['prediction'] != df['label'])]
incorrect_exp_source_no_exp_words_sample = incorrect_exp_source_no_exp_words.sample(3, random_state=30)
incorrect_exp_source_no_exp_words_sample

,Unnamed: 0,text,label,new_label,prediction
4639,4639,She's got some glass cutters\n,1,explicit_source_no_explicit_words,0
1026,1026,Haha its a shame we cant.. being the internet...,1,explicit_source_no_explicit_words,0
1262,1262,Bangin' like a drum,1,explicit_source_no_explicit_words,0


In [ ]:
for sent in incorrect_exp_source_no_exp_words_sample['text']:
  print(sent)

She's got some glass cutters

Haha its a shame we cant..  being the internet n all that 

Bangin' like a drum


In [ ]:
for sent in incorrect_exp_source_no_exp_words_sample['text']:
  perturb_sent(sent)
  print()

sentence: She's got some glass cutters

 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.948     |     0.052     |       0.0        |
  she    |     0.906     |     0.094     |      -0.042      |
   '     |     0.934     |     0.066     |      -0.014      |
   s     |     0.924     |     0.076     |      -0.025      |
  got    |     0.904     |     0.096     |      -0.045      |
  some   |     0.941     |     0.059     |      -0.007      |
 glass   |     0.91      |     0.09      |      -0.038      |
 cutter  |     0.748     |     0.252     |       -0.2       |
  ##s    |     0.907     |     0.093     |      -0.042      |

sentence: Haha its a shame we cant..  being the internet n all that 

 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.931     |     0.069     |       0.0        |
   ha    |     0.936     |     0.064     |      0.005       |
  ##ha   |     0.94      |     0.06      |      0.009       |
  its    |     0.908 

In [ ]:
# check when label is 0 and prediction is 1, with explicit text
label_0_pred_1 = df[(df['label'] == 0) & (df['prediction'] == 1) & (df['new_label'] == 'control_source_has_explicit_words')]
label_0_pred_1_sample = label_0_pred_1.sample(3, random_state=190)
label_0_pred_1_sample

,Unnamed: 0,text,label,new_label,prediction
279,279,Masturbate and grunt loudly,0,control_source_has_explicit_words,1
3528,3528,I cannot fucking stand his little PRANCE!!!!! ...,0,control_source_has_explicit_words,1
4461,4461,Always goin for da pussy,0,control_source_has_explicit_words,1


In [ ]:
for sent in label_0_pred_1_sample['text']:
  print(sent)

Masturbate and grunt loudly
I cannot fucking stand his little PRANCE!!!!! Stop
Always goin for da pussy


In [ ]:
for sent in label_0_pred_1_sample['text']:
  perturb_sent(sent)
  print()


sentence: Masturbate and grunt loudly
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.007     |     0.993     |       0.0        |
  mast   |     0.045     |     0.955     |      0.038       |
  ##ur   |     0.018     |     0.982     |      0.011       |
 ##bate  |     0.077     |     0.923     |       0.07       |
  and    |     0.047     |     0.953     |       0.04       |
 grunt   |     0.01      |     0.99      |      0.003       |
 loudly  |     0.017     |     0.983     |       0.01       |

sentence: I cannot fucking stand his little PRANCE!!!!! Stop
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.13      |     0.87      |       0.0        |
   i     |     0.168     |     0.832     |      0.038       |
 cannot  |     0.381     |     0.619     |      0.251       |
fucking  |     0.076     |     0.924     |      -0.054      |
 stand   |     0.287     |     0.713     |      0.157       |
  his    |     0.004     |     0

In [ ]:
# check when label is 0 and prediction is 1, without explicit text
label_0_pred_1_no_exp = df[(df['label'] == 0) & (df['prediction'] == 1) & (df['new_label'] == 'control_source_no_explicit_words')]
label_0_pred_1_no_exp_sample = label_0_pred_1_no_exp.sample(3, random_state=100)
label_0_pred_1_no_exp_sample

,Unnamed: 0,text,label,new_label,prediction
200,200,why do u have the day off?,0,control_source_no_explicit_words,1
3116,3116,To pump the blood throughout your body,0,control_source_no_explicit_words,1
299,299,"same, i just wanna go hommmee",0,control_source_no_explicit_words,1


In [ ]:
for sent in label_0_pred_1_no_exp_sample['text']:
  perturb_sent(sent)
  print()

sentence: why do u have the day off?
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.242     |     0.758     |       0.0        |
  why    |     0.406     |     0.594     |      0.165       |
   do    |     0.319     |     0.681     |      0.077       |
   u     |     0.472     |     0.528     |      0.231       |
  have   |     0.305     |     0.695     |      0.064       |
  the    |     0.396     |     0.604     |      0.154       |
  day    |     0.318     |     0.682     |      0.076       |
  off    |     0.701     |     0.299     |       0.46       |
   ?     |     0.575     |     0.425     |      0.334       |

sentence: To pump the blood throughout your body
  token    | class 0 proba | class 1 proba | change (class 0) |
 unmasked  |     0.015     |     0.985     |       0.0        |
    to     |     0.221     |     0.779     |      0.206       |
   pump    |     0.221     |     0.779     |      0.206       |
   the     |     0.012     |     0.98

In [ ]:
# now just take some where the prediction is 1
pred_1 = df[(df['prediction'] == 1) & (df['text'].str.split().str.len() < 15) & (df['new_label'].isin(['explicit_source_no_explicit_words','control_source_no_explicit_words']))]
pred_1_sample = pred_1.sample(10, random_state=2000)
pred_1_sample

,Unnamed: 0,text,label,new_label,prediction
754,754,"Would love to watch that, how can I make it ha...",1,explicit_source_no_explicit_words,1
2337,2337,whip it good!\n,1,explicit_source_no_explicit_words,1
238,238,I wish I had that hope also.,0,control_source_no_explicit_words,1
2082,2082,"Turn around, let’s see those curves on the bac...",1,explicit_source_no_explicit_words,1
1837,1837,All that’s missing is my tongue buried in it,1,explicit_source_no_explicit_words,1
4556,4556,Try me for size,1,explicit_source_no_explicit_words,1
2008,2008,Are you going to answer the door like that? Lol,1,explicit_source_no_explicit_words,1
2942,2942,Abuse those tight holes,1,explicit_source_no_explicit_words,1
4405,4405,Btw I love a women who swallows! ;),1,explicit_source_no_explicit_words,1
4429,4429,Amazing! I'd love to see more 😍,1,explicit_source_no_explicit_words,1


In [ ]:
for text in pred_1_sample['text']:
  perturb_sent(text)
  print()

sentence: Would love to watch that, how can I make it happen?
 token   | class 0 proba | class 1 proba | change (class 0) |
unmasked |     0.019     |     0.981     |       0.0        |
 would   |     0.084     |     0.916     |      0.064       |
  love   |     0.588     |     0.412     |      0.569       |
   to    |     0.038     |     0.962     |      0.019       |
 watch   |     0.002     |     0.998     |      -0.017      |
  that   |     0.008     |     0.992     |      -0.012      |
   ,     |     0.023     |     0.977     |      0.004       |
  how    |     0.021     |     0.979     |      0.001       |
  can    |     0.058     |     0.942     |      0.039       |
   i     |     0.07      |     0.93      |       0.05       |
  make   |     0.033     |     0.967     |      0.013       |
   it    |     0.031     |     0.969     |      0.011       |
 happen  |     0.12      |     0.88      |       0.1        |
   ?     |     0.104     |     0.896     |      0.084       |

sentenc